<a href="https://colab.research.google.com/github/douglasbarbosadelima/Data-Science/blob/master/stock_lstm_multv_res.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow==2.1.0 

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 3.9MB 53.5MB/s 
     |████████████████████████████████| 450kB 52.9MB/s 
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorflow 1.15.0
    Uninstalling tensorflow-1.15.0:
      Successfully uninstalled tensorflow-1.15.0


In [5]:
import tensorflow as tf
import pandas as pd
import math
import numpy as np
import random

tf.__version__

'2.1.0'

In [7]:
df=pd.read_csv('krot3.CSV',delimiter=';')
df.head()

,Data,Abertura,Fechamento
0,07/01/2016,14.13,14.28
1,07/04/2016,14.40,14.08
2,07/05/2016,13.93,13.85
3,07/06/2016,13.90,13.57
4,07/07/2016,13.70,13.56


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 177 entries, 0 to 176
Data columns (total 3 columns):
Data          177 non-null object
Abertura      177 non-null float64
Fechamento    177 non-null float64
dtypes: float64(2), object(1)
memory usage: 4.3+ KB


In [0]:
ind=df[df['Abertura']==0.0].index#há dias com movimentação 0.0, vamos eliminá-los

In [0]:
df.drop(ind,inplace=True)

In [11]:
df[df['Abertura']==0.0]#checking is always good

,Data,Abertura,Fechamento


Criando uma nova feature ('Med_ant'), que traz a média dos últimos 5 dias (obviamente poderia (e deveria) ser, de fato, uma totalmente nova feature, o valor do índice de outra bolsa, por exemplo). Começaremos o dataset no sexto dia (para já ter média).


In [0]:
df['Med_ant']=df['Fechamento'].rolling(5).mean()

In [13]:
df.drop(index=[0,1,2,3,4],inplace=True)
df.head()

,Data,Abertura,Fechamento,Med_ant
5,07/08/2016,13.63,14.12,13.836
6,07/11/2016,14.25,14.12,13.844
7,07/12/2016,14.12,14.63,14.000
8,07/13/2016,14.52,15.03,14.292
9,07/14/2016,15.20,15.45,14.670


Agora vamos criar sequências de 10  e médias, o target será o 11 fechamento

In [15]:
n=len(df)
print(n)
X=[]
y=[]
for i in range(n-10):
  seq=[]
  for j in range(10):
    seq.append([df.iloc[i+j,2],df.iloc[i+j,3]])
  X.append(seq)
  y.append([df.iloc[i+10,2]])
X=np.array(X)
y=np.array(y)


X[:3],y[:3]

156


(array([[[14.12 , 13.836],
         [14.12 , 13.844],
         [14.63 , 14.   ],
         [15.03 , 14.292],
         [15.45 , 14.67 ],
         [15.64 , 14.974],
         [15.8  , 15.31 ],
         [15.74 , 15.532],
         [14.95 , 15.516],
         [14.88 , 15.402]],
 
        [[14.12 , 13.844],
         [14.63 , 14.   ],
         [15.03 , 14.292],
         [15.45 , 14.67 ],
         [15.64 , 14.974],
         [15.8  , 15.31 ],
         [15.74 , 15.532],
         [14.95 , 15.516],
         [14.88 , 15.402],
         [15.08 , 15.29 ]],
 
        [[14.63 , 14.   ],
         [15.03 , 14.292],
         [15.45 , 14.67 ],
         [15.64 , 14.974],
         [15.8  , 15.31 ],
         [15.74 , 15.532],
         [14.95 , 15.516],
         [14.88 , 15.402],
         [15.08 , 15.29 ],
         [14.81 , 15.092]]]), array([[15.08],
        [14.81],
        [14.55]]))

In [0]:
tf.random.set_seed(51)
model=tf.keras.Sequential()
model.add(tf.keras.layers.LSTM(8,input_shape=(None,2)))
model.add(tf.keras.layers.Dense(1))

In [0]:
model.compile(loss='mean_squared_error', optimizer='adam')

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8)                 352       
_________________________________________________________________
dense (Dense)                (None, 1)                 9         
Total params: 361
Trainable params: 361
Non-trainable params: 0
_________________________________________________________________


In [0]:
hist=model.fit(X, y,validation_split=0.3, epochs=1000)

In [20]:
w=model.get_weights()
w

[array([[ 0.10957237,  0.06134909,  0.25267434,  0.1773188 ,  0.00763501,
         -0.13014229,  0.3617923 , -0.03316885, -0.2834658 , -0.02324965,
         -0.28621373,  0.36705825,  0.15234555,  0.17210239,  0.20950626,
          0.26297536,  0.20835884, -0.12600145,  0.28493813, -0.03576471,
          0.2438067 , -0.3257933 , -0.02139801, -0.1354632 ,  0.4874532 ,
          0.21686278,  0.26650423,  0.30045718, -0.05021737,  0.255187  ,
          0.09486912,  0.31639656],
        [ 0.2735822 ,  0.12258939,  0.02638534,  0.08224898,  0.12323064,
          0.07620972,  0.46843588, -0.03396611,  0.3004362 ,  0.20175895,
          0.35549825, -0.1638714 , -0.10700321, -0.06254163, -0.06267677,
         -0.07050116, -0.3162354 ,  0.47325867, -0.24039213, -0.05148578,
         -0.08572703, -0.17056936, -0.4673686 ,  0.02304766, -0.23116569,
          0.01638751,  0.2781282 , -0.01625872,  0.3056232 ,  0.17581676,
          0.13411315,  0.28428632]], dtype=float32),
 array([[-0.18607056,  

Repetindo o problema sem a nova feature

In [21]:
n=len(df)
print(n)
X1=[]
y1=[]
for i in range(n-10):
  seq=[]
  for j in range(10):
    seq.append([df.iloc[i+j,2]])
  X1.append(seq)
  y1.append([df.iloc[i+10,2]])
X1=np.array(X1)
y1=np.array(y1)


X1[:3],y1[:3]

156


(array([[[14.12],
         [14.12],
         [14.63],
         [15.03],
         [15.45],
         [15.64],
         [15.8 ],
         [15.74],
         [14.95],
         [14.88]],
 
        [[14.12],
         [14.63],
         [15.03],
         [15.45],
         [15.64],
         [15.8 ],
         [15.74],
         [14.95],
         [14.88],
         [15.08]],
 
        [[14.63],
         [15.03],
         [15.45],
         [15.64],
         [15.8 ],
         [15.74],
         [14.95],
         [14.88],
         [15.08],
         [14.81]]]), array([[15.08],
        [14.81],
        [14.55]]))

In [0]:
tf.random.set_seed(51)
model2=tf.keras.Sequential()
model2.add(tf.keras.layers.LSTM(8,input_shape=(None,1)))#input único em t
model2.add(tf.keras.layers.Dense(1))

In [23]:
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 8)                 320       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 9         
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [0]:
model2.compile(loss='mean_squared_error', optimizer='adam')

In [0]:
hist=model2.fit(X1, y1,validation_split=0.3, epochs=1000)

In [26]:
w2=model2.get_weights()
w2

[array([[ 0.06705458, -0.00251489,  0.377469  ,  0.34117937,  0.07241458,
         -0.11005489,  0.33612704, -0.00828004, -0.12076621,  0.01069934,
         -0.03888707,  0.37445697,  0.16426572,  0.15627412,  0.06351419,
          0.32219183, -0.02085061,  0.0746863 ,  0.2828251 , -0.27890027,
          0.09382036, -0.33705524, -0.13233455, -0.04507983,  0.4047462 ,
          0.18878525,  0.39629376,  0.23357396,  0.11056178,  0.5158412 ,
          0.11938298,  0.54915595]], dtype=float32),
 array([[-0.28955665,  0.0150408 , -0.08454219, -0.15616909,  0.19795956,
          0.2533873 ,  0.10402298, -0.05608116, -0.2700099 , -0.323385  ,
          0.14297312,  0.35444084,  0.18771288,  0.30471864, -0.16262162,
         -0.41968536, -0.272487  ,  0.03658149, -0.16183718, -0.04924168,
         -0.26480925, -0.25822768, -0.2416467 ,  0.10698018, -0.65234196,
         -0.43328252, -0.03666782, -0.09550963, -0.64173377,  0.02198145,
         -0.78102094,  0.06884123],
        [ 0.4457683 ,  